In [1]:
#Package needed for next code block
!pip3 install PyDrive

     |████████████████████████████████| 993kB 6.3MB/s 
  Stored in directory: /root/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built PyDrive


In [0]:
!pip3 install tensorboardcolab

In [0]:
#Code to authorize Colab to access Google Drive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
#Code for reading the data from Google Drive
#Non-augmented train data
download = drive.CreateFile({'id': '1-3R9ZN-yRXLmsm4oGmmuHM_REUwHEen3'})
download.GetContentFile('traindata.npy')

#Augmented train data
download = drive.CreateFile({'id': '1VOUu67nZayC63KJ6DHvBYdYbtJVwgip2'})
download.GetContentFile('augmented_data_0.npy')
download = drive.CreateFile({'id': '1gtxZ_2e_OQKXppo0CQ0VBSYKDvfN2lWN'})
download.GetContentFile('augmented_data_1.npy')
download = drive.CreateFile({'id': '1EuUwJ_2zItjKpMx9EFfEXqeUzN6YcKay'})
download.GetContentFile('augmented_data_2.npy')
download = drive.CreateFile({'id': '1DspGBsieJRYI88EGbc_ctqIMKNX7mdN5'})
download.GetContentFile('augmented_data_3.npy')
download = drive.CreateFile({'id': '1mxI_xtHzovhgAKWVFQwDxT5623ZSgDU9'})
download.GetContentFile('augmented_data_4.npy')
download = drive.CreateFile({'id': '18hPceYSPrrmnnBo9GDO5NLWjBX6Vw2r7'})
download.GetContentFile('augmented_data_5.npy')
download = drive.CreateFile({'id': '1N__M6BKzQoU-56CcePPv6tAXRb5fT6N4'})
download.GetContentFile('augmented_data_6.npy')

In [22]:
import tensorboardcolab


NameError: ignored

In [1]:
#@title  { form-width: "250px" }
from __future__ import print_function, division
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Conv2D
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.preprocessing.image import load_img, img_to_array, array_to_img
from keras.callbacks import TensorBoard
from PIL import Image
import PIL.ImageOps
import matplotlib.pyplot as plt
import os
import shutil
import glob, os
from PIL import Image
import PIL.ImageOps
import matplotlib.pyplot as plt
import sys
import numpy as np

PREPARE_COLAB_DATA = False
RUN_ON_COLAB = True
NPY_SAVEFILE = 'traindata.npy'
IMAGE_DIR = 'images/'
TRAIN_ON_AUGMENTED = False
AUGMENTED_FILES = ['../augmented_data_0.npy', '../augmented_data_1.npy', '../augmented_data_2.npy',
 '../augmented_data_3.npy', '../augmented_data_4.npy', '../augmented_data_5.npy', '../augmented_data_6.npy', ]

EPOCHS = 30000
BATCH_SIZE = 16
SAMPLE_INTERVAL = 20

class GAN():
    def __init__(self):
        self.channels = 1
        self.latent_dim = 100
        rescale_factor = 32

        optimizer = Adam(0.0001, 0.5)

        self.logdir = "./logs"
        try: 
          os.mkdir(self.logdir)
          print('Created log directory...')
        except:
          print('Log directory already exists!')

        # Empty any old log directory
        for the_file in os.listdir(self.logdir):
            file_path = os.path.join(self.logdir, the_file)
            try:
                if os.path.isfile(file_path):
                    os.unlink(file_path)
                elif os.path.isdir(file_path): shutil.rmtree(file_path)
            except Exception as e:
                print(e)

        # Empty the generated image directory
        for the_file in os.listdir("./images"):
            file_path = os.path.join("./images", the_file)
            try:
                if os.path.isfile(file_path):
                    os.unlink(file_path)
                elif os.path.isdir(file_path): shutil.rmtree(file_path)
            except Exception as e:
                print(e)


        # Load the dataset
        filelist = glob.glob("./source_imgs/*.jpg")
        imgs = [Image.open(fname) for fname in filelist]
        if RUN_ON_COLAB:
            try:
                os.mkdir(IMAGE_DIR)
                print("Created output images directory...")
            except:
                print("Output images directory already exists!")
            if TRAIN_ON_AUGMENTED:
                self.X_train = np.load(AUGMENTED_FILES[0])
            else:
                self.X_train = np.load(NPY_SAVEFILE)
            print(self.X_train.shape)
            target_size = (max([x.shape[1] for x in self.X_train]), max([x.shape[0] for x in self.X_train]))
            self.img_shape = (target_size[1], target_size[0], self.channels)
        else:
            # Load the dataset
            filelist = glob.glob("./source_imgs/*.jpg")
            imgs = [Image.open(fname) for fname in filelist]

            self.target_size  = (max([x.size[0] for x in imgs]),
                                 max([x.size[1] for x in imgs]))

            self.target_size = tuple([x//rescale_factor for x in self.target_size])

            self.X_train = []

            for img in imgs:
                old_size = img.size
                ratio = min(self.target_size[0]/old_size[0],
                            self.target_size[1]/old_size[1])

                new_size = tuple([int(x*ratio) for x in old_size])
                img = img.resize(new_size, Image.ANTIALIAS)
                img = PIL.ImageOps.invert(img)
                new_img = Image.new("L", self.target_size)
                new_img.paste(img, ((self.target_size[0]-new_size[0])//2,
                                    (self.target_size[1]-new_size[1])//2))
                self.X_train.append(new_img)

            self.X_train = np.stack(self.X_train)

            self.img_shape = (self.target_size[1],
                              self.target_size[0],
                              self.channels)

            # Rescale -1 to 1
            self.X_train = self.X_train / 127.5 - 1.
            self.X_train = np.expand_dims(self.X_train, axis=3)

        if PREPARE_COLAB_DATA:
            np.save(NPY_SAVEFILE, self.X_train)
            #quit()

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)


    def build_generator(self):

        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))

        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))

        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))

        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()


        model.add(Conv2D(16, (5, 5),
                         activation='relu',
                         padding='same',
                         input_shape=self.img_shape))

        model.add(Conv2D(8, (7, 7),
                         activation='relu',
                         padding='same',
                         input_shape=self.img_shape))

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=1, sample_interval=50):

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        tensorboard = TensorBoard(log_dir=self.logdir)
        tensorboard.set_model(self.discriminator)

        for idx in range(0, len(AUGMENTED_FILES)):
            for epoch in range(epochs):

                # ---------------------
                #  Train Discriminator
                # ---------------------

                # Select a random batch of images
                idx = np.random.randint(0, self.X_train.shape[0], batch_size)
                imgs = self.X_train[idx]

                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

                # Generate a batch of new images
                gen_imgs = self.generator.predict(noise)

                if epoch == 0 or accuracy < 80:
                    # Train the discriminator
                    d_loss_real = self.discriminator.train_on_batch(imgs, valid)
                    d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
                    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
                else:
                    # Test the discriminator
                    d_loss_real = self.discriminator.test_on_batch(imgs, valid)
                    d_loss_fake = self.discriminator.test_on_batch(gen_imgs, fake)
                    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                accuracy = 100*d_loss[1]

                # ---------------------
                #  Train Generator
                # ---------------------

                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

                if epoch == 0 or accuracy > 20:
                    # Train the generator (to have the discriminator label samples as valid)
                    g_loss = self.combined.train_on_batch(noise, valid)
                else:
                    # Train the generator (to have the discriminator label samples as valid)
                    g_loss = self.combined.test_on_batch(noise, valid)

                tensorboard.on_epoch_end(epoch, {'generator loss': g_loss, 'discriminator loss': d_loss[0], 'Accuracy': accuracy})

                # Plot the progress
                if RUN_ON_COLAB:
                    if (epoch % 200) == 0:
                        print(f"{epoch} [D loss: {d_loss[0]}, " +
                      f"acc.: {accuracy}%] [G loss: {g_loss}]")
                else:
                    print(f"{epoch} [D loss: {d_loss[0]}, " +
                      f"acc.: {accuracy}%] [G loss: {g_loss}]")

                # If at save interval => save generated image samples
                if epoch % sample_interval == 0:
                    self.sample_images(epoch)
            self.X_train = np.load(AUGMENTED_FILES[idx])
        tensorboard.on_train_end()

    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images from [-1, 1] to [1, 0] (invert)
        gen_imgs = -0.5 * gen_imgs - 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig(IMAGE_DIR+"%d.png" % epoch)
        plt.close()


if __name__ == '__main__':
    gan = GAN()
    gan.train(epochs=EPOCHS, batch_size=BATCH_SIZE, sample_interval=SAMPLE_INTERVAL)


Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Log directory already exists!
Output images directory already exists!
(510, 75, 96, 1)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 75, 96, 16)        416       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 75, 96, 8)         6280      
_________________________________________________________________
flatten_1 (Flatten)          (None, 57600)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               29491712  
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)           

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Instructions for updating:
Use tf.cast instead.
0 [D loss: 0.9100474715232849, acc.: 34.375%] [G loss: 0.029758142307400703]
200 [D loss: 0.7344865202903748, acc.: 78.125%] [G loss: 7.555872917175293]
400 [D loss: 0.19603601098060608, acc.: 96.875%] [G loss: 6.834625720977783]
600 [D loss: 0.25728583335876465, acc.: 90.625%] [G loss: 5.580780029296875]
800 [D loss: 0.3726247549057007, acc.: 81.25%] [G loss: 4.761806964874268]
1000 [D loss: 0.27107375860214233, acc.: 90.625%] [G loss: 5.047158241271973]
1200 [D loss: 0.20135806500911713, acc.: 100.0%] [G loss: 5.906414985656738]
1400 [D loss: 0.5352913737297058, acc.: 62.5%] [G loss: 6.928520679473877]
1600 [D loss: 0.20185494422912598, acc.: 96.875%] [G loss: 6.469662189483643]
1800 [D loss: 0.1314624547958374, acc.: 93.75%] [G loss: 7.300889492034912]
2000 [D loss: 0.10830563306808472, acc.: 100.0%] [G loss: 2.5080909729003906]
2200 [D loss: 0.24941760301589966, acc.: 93.75%] [G loss: 6.008194923400879]
2400 [D loss: 0.180803447961807

KeyboardInterrupt: ignored